In [1]:
# Imports

In [9]:
from Ian import client_functions, pd, np, clusters, equal_in, Benchmarks

In [10]:
pd.options.display.max_columns=200
pd.options.display.max_rows=1000
pd.options.display.max_colwidth=500

# 1. Connection

In [11]:
user = 'JM025575'
password = 'Bonniemarie120288!'
client = "IHC_UT"
i = client_functions(**{
    'host' : clusters('Deer'),
    'user' : f'{user}',
    'password' : f'{password}',
    'schema' : 'INTERMOUNTAIN',
    'population_id' : 'e9a85078-ff1f-40b0-9894-402e56c4b679'
})

### Test

In [12]:
# test connection works (should return current date)
i.query_df(f"""
SELECT GETDATE()
""")

,GETDATE
0,2019-11-22 10:26:19.124916


## 2.Query Data

In [13]:
# for 3 months in specified querter, put 1st date of each into an array
#year_quarter = '2019Q2'
dates = pd.DatetimeIndex(['2019-09-01', '2019-10-01', '2019-11-01'])

In [14]:
dates

DatetimeIndex(['2019-09-01', '2019-10-01', '2019-11-01'], dtype='datetime64[ns]', freq=None)

In [15]:
# specify the registries of interest
registries = [
    'atrisk'
]

In [16]:
# create Benchmarks query
#scorable>False dashboard_histories table, True>scorecard_trending
q = Benchmarks.query(schema=i.schema, 
                     dream_dates=dates, 
                     scorable = False,  
                     registries=registries)

In [17]:
print(q)


        SELECT DISTINCT
            'INTERMOUNTAIN' AS client,
            q.*
        FROM
            (
            SELECT DISTINCT
                st.created_dt_tm AS date,
                m.fully_qualified_name,
                st.dashboard_history_personnel_id AS prsnl_id,
                m.composite_percentage_met_count AS numerator,
                (m.composite_percentage_met_count + m.composite_percentage_not_met_count) AS denominator
            FROM
                INTERMOUNTAIN.PH_F_Dashboard_History_Trending_Registry_Measure m
                JOIN INTERMOUNTAIN.PH_F_Dashboard_History_Trending_Registry r ON (
                    m.registry_trending_uid = r.registry_trending_uid
                    AND m.dashboard_history_uuid = r.dashboard_history_uuid
                )
                JOIN INTERMOUNTAIN.PH_F_Dashboard_History_Trending_Registry_Mara_Risk_Score s ON (
                    r.registry_trending_uid = s.mara_risk_score_uuid
                    AND r.dashboard_his

In [10]:
scorable = i.query_df(q)

In [11]:
scorable.head()

,client,population_id,date,prsnl_id,fully_qualified_name,numerator,denominator
0,INTERMOUNTAIN,e9a85078-ff1f-40b0-9894-402e56c4b679,2019-09-02 01:50:22.440000-05:00,01b8c101-28f6-4618-8c53-bdb3426e3bd2,intermountain.atrisk.clinical/blood-pressure-lt-140-90-mm-hg,2,7
1,INTERMOUNTAIN,e9a85078-ff1f-40b0-9894-402e56c4b679,2019-09-02 01:50:22.440000-05:00,01b8c101-28f6-4618-8c53-bdb3426e3bd2,intermountain.atrisk.clinical/breast-cancer-screening,10,10
2,INTERMOUNTAIN,e9a85078-ff1f-40b0-9894-402e56c4b679,2019-09-02 01:50:22.440000-05:00,01b8c101-28f6-4618-8c53-bdb3426e3bd2,intermountain.atrisk.clinical/colorectal-cancer-screening,9,18
3,INTERMOUNTAIN,e9a85078-ff1f-40b0-9894-402e56c4b679,2019-09-02 01:50:22.440000-05:00,01b8c101-28f6-4618-8c53-bdb3426e3bd2,intermountain.atrisk.clinical/depression-screening,4,43
4,INTERMOUNTAIN,e9a85078-ff1f-40b0-9894-402e56c4b679,2019-09-02 01:50:22.440000-05:00,01b8c101-28f6-4618-8c53-bdb3426e3bd2,intermountain.atrisk.clinical/diabetic-eye-exam,0,2


## 3. Clean Data

In [12]:
reg_meas_names = i.query_df(f"""
SELECT DISTINCT
mo.fully_qualified_name,
INITCAP( REPLACE( SUBSTR( mo.program_id, INSTR( mo.program_id, '.', -1 ) + 1, CHAR_LENGTH( mo.program_id ) ), '-', ' ' )) AS registry,
INITCAP( REPLACE( SUBSTR( mo.fully_qualified_name, INSTR( mo.fully_qualified_name, '/' ) + 1, CHAR_LENGTH( mo.fully_qualified_name )),'-',' ')) AS "measure"
FROM {i.schema}.PH_F_Measure_Outcome mo
JOIN {i.schema}.PH_F_Scorecard_Trending_Personnel_Point sc on(
    mo.program_id = sc.program_id)
""")

In [13]:
reg_meas_names.head()

,fully_qualified_name,registry,measure
0,cernerstandard.rheumatoidarthritis.clinical/rheumatoid-arthritis-management-measure,Rheumatoid Arthritis,Rheumatoid Arthritis Management Measure
1,intermountain.pediatricwellness.clinical/well-child-visits-in-the-first-15-months-of-life,Pediatric Wellness,Well Child Visits In The First 15 Months Of Life
2,intermountain.hypertension.clinical/bp-less-than-140-90,Hypertension,Bp Less Than 140 90
3,cernerstandard.candaimmunizations.clinical/pneumococcal-vaccination-pediatric,C And A Immunizations,Pneumococcal Vaccination Pediatric
4,cernerstandard.candaimmunizations.clinical/human-papillomavirus-vaccination,C And A Immunizations,Human Papillomavirus Vaccination


In [14]:
# join to get Registry & Measure
new_scorable = scorable.merge(reg_meas_names,on="fully_qualified_name")
# add year_quarter
#new_scorable["year_quarter"] = dates
# reorder columns
new_scorable = new_scorable[[
    "fully_qualified_name",
    "registry",
    "measure",
    "date",
    "prsnl_id",
    "numerator",
    "denominator"
]]

In [15]:
new_scorable.head(1)

,fully_qualified_name,registry,measure,date,prsnl_id,numerator,denominator
0,intermountain.atrisk.clinical/blood-pressure-lt-140-90-mm-hg,Atrisk,Blood Pressure Lt 140 90 Mm Hg,2019-09-02 01:50:22.440000-05:00,01b8c101-28f6-4618-8c53-bdb3426e3bd2,2,7


In [16]:
new_scorable.to_csv(f"{client}_scorable_raw_data_Q42019_AtRisk.csv", index=False)